In [1]:
from tqdm.notebook import tqdm
import math
import gym
import torch
import torch.optim as optim 
from torch.utils.tensorboard import SummaryWriter
from collections import deque

from active_rl.networks.dqn_atari import ENS_DQN_Large
from active_rl.utils.memory import ReplayMemory
from active_rl.utils.optimization import std_opt_end_independent
from active_rl.environments.atari_wrappers import make_atari, wrap_deepmind
from active_rl.utils.atari_utils import fp, evaluate
from active_rl.utils.acquisition_functions import ens_random
from active_rl.utils.action_selection import ProbabilisticActionSelector

In [2]:
env_name = 'Breakout'
env_raw = make_atari('{}NoFrameskip-v4'.format(env_name))
env = wrap_deepmind(env_raw, frame_stack=False, episode_life=True, clip_rewards=True)
c,h,w = c,h,w = fp(env.reset()).shape
n_actions = env.action_space.n

In [3]:
BATCH_SIZE = 64
LR = 0.0000625
GAMMA = 0.99
EPS_START = 1.
EPS_END = 0.01
EPS_DECAY = 1000000 
NUM_STEPS = 20000000
POLICY_UPDATE = 4
TARGET_UPDATE = 4000
EVALUATE_FREQ = 10000
MEMORY_CAPACITY = 100000
INITIAL_STEPS=1000
PERCENTAGE = 0.5

NAME = 'standard_ens_large_eps_1_prob_action_selection_independent_opt'

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # if gpu is to be used
policy_net = ENS_DQN_Large(n_actions).to(device)
target_net = ENS_DQN_Large(n_actions).to(device)
policy_net.apply(policy_net.init_weights)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.Adam(policy_net.parameters(), lr=LR, eps=1.5e-4)

In [5]:
memory = ReplayMemory(MEMORY_CAPACITY, [5,h,w], n_actions, device)
action_selector = ProbabilisticActionSelector(policy_net, EPS_START, EPS_END, EPS_DECAY, n_actions, device)

In [6]:
steps_done = 0
writer = SummaryWriter(f'runs/{NAME}')

In [7]:
q = deque(maxlen=5)
done=True
eps = 0
episode_len = 0

In [8]:
progressive = tqdm(range(NUM_STEPS), total=NUM_STEPS, ncols=400, leave=False, unit='b')
for step in progressive:
  if done:
    env.reset()
    sum_reward = 0
    episode_len = 0
    img, _, _, _ = env.step(1) # BREAKOUT specific !!!
    for i in range(10): # no-op
      n_frame, _, _, _ = env.step(0)
      n_frame = fp(n_frame)
      q.append(n_frame)
        
  # Select and perform an action
  state = torch.cat(list(q))[1:].unsqueeze(0)
  action, eps, act_mean, act_var = action_selector.select_action(state)
  n_frame, reward, done, info = env.step(action)
  n_frame = fp(n_frame)

  # 5 frame as memory
  q.append(n_frame)
  memory.push(torch.cat(list(q)).unsqueeze(0), action, reward, done) # here the n_frame means next frame from the previous time step
  episode_len += 1

  # Perform one step of the optimization (on the target network)
  if step % POLICY_UPDATE == 0 and step > INITIAL_STEPS:
    loss = std_opt_end_independent(policy_net, target_net, optimizer, memory, batch_size=BATCH_SIZE)
    if loss is not None:
      writer.add_scalar('Performance/loss', loss, step)
    
  # Update the target network, copying all weights and biases in DQN
  if step % TARGET_UPDATE == 0 and step > INITIAL_STEPS:
    target_net.load_state_dict(policy_net.state_dict())
    
  if step % EVALUATE_FREQ == 0 and step > INITIAL_STEPS:
    evaluated_reward = evaluate(step, policy_net, device, env_raw, n_actions, eps=0.05, num_episode=15)
    writer.add_scalar('Performance/reward', evaluated_reward, step)
    if act_mean is not None and act_var is not None:
      writer.add_histogram('Other/action_mean', act_mean)
      writer.add_histogram('Other/action_variance', act_var)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/tony/anaconda3/envs/ml/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-238118d675e6>", line 26, in <module>
    loss = std_opt_end_independent(policy_net, target_net, optimizer, memory, batch_size=BATCH_SIZE)
  File "/mnt/hdd1/ml/Active-Reinforcement-Learning/active_rl/utils/optimization.py", line 321, in std_opt_end_independent
    n_state_batch = n_state_batch.to(device)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/tony/anaconda3/envs/ml/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most rec

KeyboardInterrupt: 